# Welcome to the Quantum Parallel Universe

### Before Running this Notebook

- Create a `token.txt` file in the same folder as this notebook
  - `token.txt` should contain a token sufficient for [`qiskit_ibm_provider.IBMProvider()`](https://github.com/Qiskit/qiskit-ibm-provider#enable-account-for-current-session) authentication.

---

### Get Token

In [1]:
token = None
with open('token.txt', 'r') as f:
  token = f.read().strip()

### Imports

In [2]:
import math
from qiskit import QuantumCircuit

## Let $N = 2^m$

### Initializer Class

In [3]:
# Stands for Quantum Parallel Initializer (the acronym is a tongue-in-cheek reference to MPI and QMPI)
class QPI:
  N = None
  qubit_list = None
  circuit = None

  def _validator(self, m: int):
    if not isinstance(m, int):
      raise TypeError("Only integer arguments accepted.")
    if m < 0:
      raise ValueError("`m` must be at least 0 (evaluated 2^m).")
    self.N = 2**m
    self.qubit_list = [i for i in range(self.N)]

  def _initCircuit(self) -> QuantumCircuit:
    c = QuantumCircuit(self.N, self.N)
    for i in self.qubit_list:
      c.reset(i)
    self.circuit = c

  def __init__(self, m: int):
    self._validator(m=m)
    self._initCircuit()

  def m(self):
    return math.ceil(math.log2(self.N))

### Generate Linear Time Complexity Circuits for $|\text{GHZ}_N\rangle$

In [4]:
def linear_complexity_GHZ(m: int) -> QPI:
  q = QPI(m=m)
  q.circuit.h(0)
  for i in range(1, q.N):
    q.circuit.cx(i-1, i)
  q.circuit.barrier()
  q.circuit.measure(*((q.qubit_list,)*2))
  return q

### Generate Logaritmic Complexity Circuits for $|\text{GHZ}_N\rangle$

In [5]:
def log_complexity_GHZ(q: QPI, m=None) -> QPI:
  if m is None:
    m = q.m()
  if m == 0:
    q.circuit.h(0)
    q.circuit.measure(*((q.qubit_list,)*2))
    q.circuit.barrier()
    return q
  else:
    q = log_complexity_GHZ(q=q, m=m-1)
    for i in q.qubit_list:
      try:
        q.circuit.cx(i-1, m-1)
      except AttributeError as e:
        print(m)
        raise e

---

In [6]:
# linear_complexity_GHZ(2).circuit.draw(output='mpl')

In [7]:
log_complexity_GHZ(q=QPI(m=2)).circuit.draw(output='mpl')

2


AttributeError: 'NoneType' object has no attribute 'circuit'